In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Experiment configuration
experiment_name = "ekf_n1000_3"
dataset_args = {
    'use_proprioception': True,
    'use_haptics': True,
    'use_vision': True,
    'vision_interval': 2,
}

In [3]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm_notebook

import fannypack
from lib import panda_datasets, panda_baseline_models, panda_baseline_training
from lib.ekf import KalmanFilterNetwork
from fannypack import utils
from lib import dpf
from lib.panda_models import PandaDynamicsModel, PandaEKFMeasurementModel



In [4]:
print("Creating dataset...")
# dataset_full = panda_datasets.PandaParticleFilterDataset(
#     'data/gentle_push_10.hdf5',
#     subsequence_length=16,
#     **dataset_args)

e2e_trainset = panda_datasets.PandaParticleFilterDataset(
    "data/gentle_push_1000.hdf5",
    subsequence_length=16,
    particle_count=1,
    particle_stddev=(.03, .03),
    **dataset_args
)



Creating dataset...
Parsed data: 13331 active, 1669 inactive
Keeping (inactive): 1669


In [5]:
# dataset_dynamics = panda_datasets.PandaDynamicsDataset(
#     'data/gentle_push_100.hdf5',
#     subsequence_length=16,
#     **dataset_args)

dataset_measurement = panda_datasets.PandaMeasurementDataset(
    'data/gentle_push_100.hdf5',
    subsequence_length=16,
    stddev=(0.5, 0.5),
    samples_per_pair=20,
    **dataset_args)

dynamics_recurrent_trainset = panda_datasets.PandaSubsequenceDataset(
    "data/gentle_push_100.hdf5",
    subsequence_length=32,
    **dataset_args
)


Loaded 24000 points


In [7]:
measurement = PandaEKFMeasurementModel(units=128)
dynamics = PandaDynamicsModel(use_particles=False)
ekf = KalmanFilterNetwork(dynamics, measurement)
print("Creating model...")
buddy = fannypack.utils.Buddy(experiment_name, 
                              ekf, 
                              optimizer_names=["ekf", "ekf_dynamics", "ekf_measurement"], 
                              load_checkpoint=True,
)


Creating model...
[buddy-ekf_n1000_3] Using device: cuda
[buddy-ekf_n1000_3] No checkpoint found


In [8]:
dataloader_measurement = torch.utils.data.DataLoader(
    dataset_measurement, batch_size=16, shuffle=True, num_workers=2, drop_last=True)
for _ in tqdm_notebook(range(3)):
    for batch_idx, batch in enumerate(tqdm_notebook(dataloader_measurement)):
        noisy_state, observation, _, state  = fannypack.utils.to_device(batch, buddy._device)
#         states = states[:,0,:]
        state_update, R = ekf.measurement_model(observation, noisy_state)
        loss = F.mse_loss(state_update, state)
        buddy.minimize(loss,
                       optimizer_name="ekf_measurement",
                       checkpoint_interval=500)

        with buddy.log_scope("ekf_measurement"):
            buddy.log("loss", loss)
            buddy.log("label_mean", fannypack.utils.to_numpy(state).mean())
            buddy.log("label_std", fannypack.utils.to_numpy(state).std())
            buddy.log("pred_mean", fannypack.utils.to_numpy(state_update).mean())
            buddy.log("pred_std", fannypack.utils.to_numpy(state_update).std())
buddy.save_checkpoint()

/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000000500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000001000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000001500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000002000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000002500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000003000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000003500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000004000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000004500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000005000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000

/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000030500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000031000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000031500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000032000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000032500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000033000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000033500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000034000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000034500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000035000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000

[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000060500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000061000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000061500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000062000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000062500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000063000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000063500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000064000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000064500.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000065000.ckpt
[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000

KeyboardInterrupt: 

In [9]:
# training dynamics model 

dataloader_dynamics = torch.utils.data.DataLoader(
    dynamics_recurrent_trainset, batch_size=32, shuffle=True, num_workers=2, drop_last=True)

log_interval=10
epoch_losses = []

for _ in tqdm_notebook(range(10)):
    for batch_idx, batch in enumerate(tqdm_notebook(dataloader_dynamics)):
        batch_gpu = utils.to_device(batch, buddy._device)
        batch_states, batch_obs, batch_controls = batch_gpu

        N, timesteps, control_dim = batch_controls.shape
        N, timesteps, state_dim = batch_states.shape
        assert batch_controls.shape == (N, timesteps, control_dim)
        
        prev_states = batch_states[:, 0, :]
    
        losses = []
        magnitude_losses = []
        direction_losses = []
        
        # Compute some state deltas for debugging
        label_deltas = np.mean(utils.to_numpy(
            batch_states[:, 1:, :] - batch_states[:, :-1, :]
        ) ** 2, axis=(0, 2))
        assert label_deltas.shape == (timesteps - 1, )
        pred_deltas = []
        
        for t in range(1, timesteps):
            controls = batch_controls[:,t, :]
            new_states = ekf.dynamics_model(
                prev_states,
                controls,
                noisy=False, 
            )
            
            pred_delta = prev_states - new_states 
            label_delta = batch_states[:, t-1, :] - batch_states[:, t, :]
            
        
            #todo: maybe switch back to l2
            timestep_loss = F.l1_loss(new_states, batch_states[:, t, :])
            
            losses.append(timestep_loss)
            
            pred_deltas.append(np.mean(
                utils.to_numpy(new_states - prev_states) ** 2
            ))
            prev_states = new_states
        
        pred_deltas = np.array(pred_deltas)
        assert pred_deltas.shape == (timesteps - 1, )

        loss = torch.mean(torch.stack(losses))
        epoch_losses.append(loss)
        buddy.minimize(
            loss,
            optimizer_name="ekf_dynamics",
            checkpoint_interval=100)

        if buddy.optimizer_steps % log_interval == 0:
            with buddy.log_scope("dynamics_recurrent"):
                buddy.log("Training loss", loss)

                buddy.log("Label delta mean", label_deltas.mean())
                buddy.log("Label delta std", label_deltas.std())

                buddy.log("Pred delta mean", pred_deltas.mean())
                buddy.log("Pred delta std", pred_deltas.std())

                if magnitude_losses:
                    buddy.log("Magnitude loss",
                              torch.mean(torch.tensor(magnitude_losses)))
                if direction_losses:
                    buddy.log("Direction loss",
                              torch.mean(torch.tensor(direction_losses)))

    print("Epoch loss:", np.mean(utils.to_numpy(epoch_losses)))
buddy.save_checkpoint()

/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.



Epoch loss: 1.3575385


[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000084700.ckpt

Epoch loss: 1.094554



Epoch loss: 0.8616328


...


Epoch loss: 0.4890482


[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000084800.ckpt

Epoch loss: 0.42679566



Epoch loss: 0.3791839



Epoch loss: 0.34170666



Epoch loss: 0.31136242

[buddy-ekf_n1000_3] Saved checkpoint to path: checkpoints/ekf_n1000_3-0000000000084872.ckpt


In [10]:
# training e2e model 

log_interval = 10
dataloader_full = torch.utils.data.DataLoader(
    e2e_trainset, batch_size=32, shuffle=True, num_workers=2, drop_last=True)
# for _ in tqdm_notebook(range(10)):



print("running")
for _ in tqdm_notebook(range(100)):
    for batch_idx, batch in enumerate(tqdm_notebook(dataloader_full)):
#     for batch_idx, batch in enumerate((dataloader_full)):
        # Transfer to GPU and pull out batch data
        batch_gpu = utils.to_device(batch, buddy._device)
        _, batch_states, batch_obs, batch_controls = batch_gpu
        # N = batch size, M = particle count
        N, timesteps, control_dim = batch_controls.shape
        N, timesteps, state_dim = batch_states.shape
        assert batch_controls.shape == (N, timesteps, control_dim)
        
        state = batch_states[:, 0, :]
        state_sigma = torch.eye(state.shape[-1], device=buddy._device)*0.001
        state_sigma = state_sigma.unsqueeze(0).repeat(N, 1, 1)
            
        # Accumulate losses from each timestep
        losses = []
        for t in range(1, timesteps-1):
            prev_state = state
            prev_state_sigma = state_sigma

            state, state_sigma = ekf.forward(
                prev_state,
                prev_state_sigma,
                utils.DictIterator(batch_obs)[:, t, :],
                batch_controls[:, t, :],
                noisy_dynamics=True
            )
            
            assert state.shape == batch_states[:, t, :].shape
            loss = torch.mean((state - batch_states[:, t, :]) ** 2)
            losses.append(loss)
        buddy.minimize(
            torch.mean(torch.stack(losses)),
            optimizer_name="ekf",
            checkpoint_interval=500)

        if buddy.optimizer_steps % log_interval == 0:
            with buddy.log_scope("ekf"):
                buddy.log("Training loss", loss)

    print("Epoch loss:", np.mean(utils.to_numpy(losses)))
buddy.save_checkpoint()



running


/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


RuntimeError: size mismatch, m1: [32 x 512], m2: [384 x 256] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:290